Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Convert Model and Containerize
* Create Workspace
* Register Demo model
* Build Image 
* Create Hub and Prepare for device and deploy
* Deploy Model

![title](aml_config/AMLimages/VAIDKe2e.png)

In [ ]:
# For prod
!pip install azureml-core azureml-contrib-iot azure-mgmt-containerregistry azure-cli
!az extension add --name azure-cli-iot-ext

In [ ]:
import os

# Ensure you are running from the correct environment
import sys
sys.executable

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)
from azureml.core import Workspace

### Create a Workspace
#### Change this cell from markdown to code and run this if you need to create a workspace 
ws=Workspace.create(subscription_id="your-subscription-ID-goes-here",
                resource_group="your-resource-group-goes-here",
                name="your-ML-workspace-name-goes-here",
                location="location-of-your-ML-workspace")
                
ws.write_config()

In [ ]:
#Initialize Workspace 
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Parameters
Enter your parameters for the next  automated steps:
- Creation of a workspace, 
- Creation of IoT Hub, 
- Device registration, 
- Demo model registration,
- Creation of Model Image for Azure IoT Edge Module deployment 
- Device Deployment setup
After this step you will need to copy your connection string and provide it to your device 

In [ ]:
# Parameter list

# Pick a name for what you want to call the module you deploy to the camera
module_name = "visionsample"

# Resource group in Azure 
resource_group_name= ws.resource_group
iot_rg="vaidk_"+resource_group_name

# Azure region where your services will be provisioned
iot_location="eastus2"

# Azure IoT Hub name
iot_hub_name="iothub-"+ ws.get_details()["name"]

# Pick a name for your camera
iot_device_id="vadik_"+ ws.get_details()["name"]

# Pick a name for the deployment configuration
iot_deployment_id="demovaidk"

# MobileNet ImageNet model

This step uses the trained model from your local folder in the Notebooks shell.
There are three files (i) the model_name.pb file, (ii) the lables_names.txt and (iii) va-snpe-engine-library_config.json, in this folder. 
This va-snpe-engine-library_config file is used by the camera when loading the model into the inference engine.

# key fields are 
Engine: This is the network used by the model
    0: MobileNet
    1: MobileNet-SSD 
    2: SqueezeNet
NetworkIO: 
    0: CPU (default)
    1: DSP
Runtime: this is the HW option to use for inferencing
    0: CPU 
    1: DSP 
    2: GPU
ConfThreshold: This is the threshold for when the bounding boxes are shown or inferencing results are shown on screen.

![title](aml_config/AMLimages/Image.png)

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path = "models/mobilenet-imagenet/",
                       model_name = "imagenet_2_frozen.pb",
                       tags = {'Device': "peabody", 'type': "mobilenet", 'area': "iot", 'version': "1.0"},
                       description = "TF SNPE quantization friendly MobileNet",
                       workspace = ws)

print(model.name, model.url, model.version, model.id, model.created_time)

In [ ]:
print(model.name, model.url, model.version, model.id, model.created_time)

In [ ]:
# You can download the model to see what was registered
# model.download()

### Convert Model to run on the Vision AI Dev Kit

In [ ]:
from azureml.contrib.iot.model_converters import SnpeConverter

# submit a compile request
compile_request = SnpeConverter.convert_tf_model(
    ws,
    source_model=model,
    input_node="input",
    input_dims="1,224,224,3",
    outputs_nodes = ["MobilenetV1/Predictions/Reshape_1"],
    allow_unconsumed_nodes = True)
print(compile_request._operation_id)

In [ ]:
# Wait for the request to complete
compile_request.wait_for_completion(show_output=True, timeout=60)

In [ ]:
# Get the converted model
converted_model = compile_request.result
print(converted_model.name, converted_model.url, converted_model.version, converted_model.id, converted_model.created_time)

In [ ]:
# You can downlaod the model to see what the conversion result was
# converted_model.download()

### Build the container image for IoT to deploy to the Vision AI Dev Kit

In [ ]:
# NEW version of main.py
from azureml.core.image import Image
from azureml.contrib.iot import IotContainerImage
print ('We will create an image for you now ...')
image_config = IotContainerImage.image_configuration(
                                 architecture="arm32v7",
                                 execution_script="main.py", 
                                 dependencies=["camera.py","iot.py","ipcprovider.py","utility.py", "frame_iterators.py"],
                                 docker_file="Dockerfile", 
                                 tags = ["mobilenet"],
                                 description = "Updated MobileNet trained on ImageNet")
image = Image.create(name = "mobilenetimagenet",
                     # this is the model object 
                     models = [converted_model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output = True)

In [ ]:
# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group_name, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

In [ ]:
file = open('./deployment-template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
print ( 'We will create your HUB now')

with open ('setsub','w+') as command1:
    command1.write('az account set --subscription'+subscription_id )
    !
    sh setsub

with open ('create','w+') as command2:
    regcommand="\n echo Installing Extension ... \naz extension add --name azure-cli-iot-ext \n"+ "\n echo CREATING RG "+iot_rg+"... \naz group create --name "+ iot_rg +" --location "+ iot_location+ "\n" +"\n echo CREATING HUB "+iot_hub_name+"... \naz iot hub create --name "+ iot_hub_name + " --resource-group "+ iot_rg +" --sku S1"
    command2.write(regcommand +"\n echo CREATING DEVICE ID "+iot_device_id+"... \n az iot hub device-identity create --device-id "+ iot_device_id + " --hub-name " +  iot_hub_name +" --edge-enabled")
    !sh create

with open ('deploy','w+')as command3:
    createcommand="\n echo DEPLOYING "+iot_deployment_id+" ... \naz iot edge deployment create --deployment-id \"" + iot_deployment_id + "\" --content \"deployment.json\" --hub-name \"" +  iot_hub_name +"\" --target-condition \"deviceId='"+iot_device_id+"'\" --priority 1"
    command3.write(createcommand)
    !sh deploy

with open ('details','w+')as command4:
    get_string="\n echo THIS IS YOUR CONNECTION STRING ... \naz iot hub device-identity show-connection-string --device-id  \"" + iot_device_id + "\" --hub-name \"" +  iot_hub_name+"\""
    command4.write(get_string)
    print("COPY THIS CONNECTION STRING FOR YOUR DEVICE")
    !sh details


## Deploy image as an IoT module

![title](aml_config/AMLimages/Deploy.png)

### Set subscription to the same as your workspace

In [ ]:
%%writefile ./setsub
az account set --subscription 

In [ ]:
iot_sub=ws.subscription_id
%store iot_sub >> setsub
!sh setsub 
print ('{}'.format(iot_sub))

### Provision a new Azure IoT Hub

In [ ]:
#RG and location to create hub
iot_rg="vaidk_"+resource_group_name
iot_location=ws.get_details()["location"]
#temp to delete
iot_location="eastus2"
iot_hub_name="iothub-"+ ws.get_details()["name"]
iot_device_id="vadik_"+ ws.get_details()["name"]
iot_deployment_id="demovaidk"
print('{}'.format(iot_hub_name))

In [ ]:
%%writefile ./create
#Command to create hub and device


In [ ]:
# Adding Intialization steps
regcommand="\n echo Installing Extension ... \naz extension add --name azure-cli-iot-ext \n"+ "\n echo CREATING RG "+iot_rg+"... \naz group create --name "+ iot_rg +" --location "+ iot_location+ "\n" +"\n echo CREATING HUB "+iot_hub_name+"... \naz iot hub create --name "+ iot_hub_name + " --resource-group "+ iot_rg +" --sku S1"
#print('{}'.format(regcommand))
%store regcommand >> create

### Create Identity for your device 

In [ ]:
#Adding Device ID 
create_device="\n echo CREATING DEVICE ID "+iot_device_id+"... \n az iot hub device-identity create --device-id "+ iot_device_id + " --hub-name " +  iot_hub_name +" --edge-enabled"
#print('{}'.format(create_device))
%store create_device >> create

In [ ]:
#Create command and configure device 
!sh create

### Create Deployment

In [ ]:
%%writefile ./deploy
#Command to create hub and device


In [ ]:
#Add deployment command
deploy_device="\n echo DEPLOYING "+iot_deployment_id+" ... \naz iot edge deployment create --deployment-id \"" + iot_deployment_id + "\" --content \"deployment.json\" --hub-name \"" +  iot_hub_name +"\" --target-condition \"deviceId='"+iot_device_id+"'\" --priority 1"
#print('{}'.format(deploy_device))
%store deploy_device >> deploy

In [ ]:
#run deployment to stage all work for when the model is ready 
!sh deploy

### Use this conenction string on your camera to Initialize it

In [ ]:
%%writefile ./showdetails
#Command to create hub and device

In [ ]:
#Add deployment command
get_string="\n echo THIS IS YOUR CONNECTION STRING ... \naz iot hub device-identity show-connection-string --device-id  \"" + iot_device_id + "\" --hub-name \"" +  iot_hub_name+"\""
#print('{}'.format(get_string))
%store get_string >> showdetails
!sh showdetails

In [ ]:
!az account set --subscription 5f08d643-1910-4a38-a7c7-84a39d4f42e0
!az iot hub show --name hub-peabody

## Remove Deployment